In [1]:
from src.generate_surfaces import ResponseFunction, six_curve_sum
import torch
from ax import Client, RangeParameterConfig
from src import BO_visuals

Using font family: Brisbane Trial


# Single prediction

## initialise response and client

In [2]:
simple_func = lambda x: sum(x)

resp = ResponseFunction(simple_func, 3)
# 1. Initialize the Client.
client = Client()



## configure the search domain

In [3]:
# 2. Configure where Ax will search.
client.configure_experiment(
    name="booth_function",
    parameters=[
        RangeParameterConfig(
            name=f"x{i}",
            bounds=(0, 20.0),
            parameter_type="float",
        ) for i in range(3)
    ],
)

In [4]:
# 3. Configure a metric for Ax to target (see other Tutorials for adding constraints,
# multiple objectives, tracking metrics etc.)
client.configure_optimization(objective="booth")

In [5]:
# 4. Conduct the experiment with 20 trials: get each trial from Ax, evaluate the
# objective function, and log data back to Ax.
for _ in range(20):
    # Use higher value of `max_trials` to run trials in parallel.
    for trial_index, parameters in client.get_next_trials(max_trials=1).items():
        coordinates = torch.tensor(
            [[parameters[f"x{i}"]] for i in range(3)], dtype=torch.float32
        )
        #print(coordinates)
        #print(resp.evaluate(coordinates)[0])

        client.complete_trial(
            trial_index=trial_index,
            raw_data={
            "booth": resp.evaluate(coordinates)[0].item()
            },
        )

# 5. Obtain the best-performing configuration; the true minimum for the booth

client.get_best_parameterization()

[INFO 09-15 12:11:44] ax.api.client: GenerationStrategy(name='Center+Sobol+MBM:fast', nodes=[CenterGenerationNode(next_node_name='Sobol'), GenerationNode(node_name='Sobol', generator_specs=[GeneratorSpec(generator_enum=Sobol, model_key_override=None)], transition_criteria=[MinTrials(transition_to='MBM'), MinTrials(transition_to='MBM')]), GenerationNode(node_name='MBM', generator_specs=[GeneratorSpec(generator_enum=BoTorch, model_key_override=None)], transition_criteria=[])]) chosen based on user input and problem structure.
[INFO 09-15 12:11:44] ax.api.client: Generated new trial 0 with parameters {'x0': 10.0, 'x1': 10.0, 'x2': 10.0} using GenerationNode CenterOfSearchSpace.
[INFO 09-15 12:11:44] ax.api.client: Trial 0 marked COMPLETED.
[INFO 09-15 12:11:44] ax.api.client: Generated new trial 1 with parameters {'x0': 16.371737, 'x1': 16.252646, 'x2': 15.461453} using GenerationNode Sobol.
[INFO 09-15 12:11:44] ax.api.client: Trial 1 marked COMPLETED.
[INFO 09-15 12:11:44] ax.api.client

({'x0': 20.0, 'x1': 20.0, 'x2': 20.0}, {'booth': (60.0, nan)}, 6, '6_0')

In [ ]:
import pandas as pd
from BO_visuals import GPVisualiser


print("Results DataFrame:")
print(GPVisualiser.get_obs_from_client(client))
results = GPVisualiser.get_obs_from_client(client)

Results DataFrame:
           x0         x1         x2   response
0   10.000000  10.000000  10.000000  30.000000
1   16.371737  16.252646  15.461453  48.085835
2    8.886926   2.314023   2.113169  13.314117
3    0.424228  12.514733  11.412951  24.351912
4   12.900493   8.566431   6.011215  27.478140
5   20.000000  19.774771  19.404006  59.178780
6   20.000000  20.000000  20.000000  60.000000
7   20.000000  12.971247  20.000000  52.971245
8   20.000000  20.000000  16.878801  56.878799
9   14.385030  11.755283  16.173646  42.313961
10  18.597959  20.000000  20.000000  58.597961
11   2.905853   6.873872   4.201231  13.980956
12   7.941221  16.807326  13.507313  38.255859
13  19.430219   1.770560   6.730831  27.931610
14  15.146318  18.257415   1.852409  35.256142
15   7.250461   3.142737  18.551841  28.945040
16   2.215080  13.194712   9.088430  24.498222
17  10.101122   8.237178  11.140025  29.478325
18  12.119859  15.109707   8.333256  35.562820
19   0.484431   0.072182  11.282115  11.8

In [7]:
from math import floor
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll_torch
from gpytorch.mlls import ExactMarginalLogLikelihood
from torch._tensor import Tensor
import importlib
import math
import seaborn as sns

import matplotlib.pyplot as plt

In [8]:
%matplotlib
%matplotlib notebook
%matplotlib inline

Using matplotlib backend: module://matplotlib_inline.backend_inline


In [ ]:
importlib.reload(GPVisualiser)  # Reload to get latest changes
importlib

<module 'importlib' from 'C:\\Users\\326po\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\importlib\\__init__.py'>

In [ ]:
import plotly.graph_objects as go

def plt_gp_along_dimension_plotly(grid: Tensor, mean: Tensor, std: Tensor, results: pd.DataFrame, dim_name: str, coordinates, point_size):
    """Plot GP mean and confidence intervals along a single dimension using Plotly."""
    fig = go.Figure()

    # GP mean line
    fig.add_trace(go.Scatter(
        x=grid.numpy(),
        y=mean,
        mode='lines',
        name='GP mean'
    ))

    # Confidence interval
    fig.add_trace(go.Scatter(
        x=np.concatenate([grid.numpy(), grid.numpy()[::-1]]),
        y=np.concatenate([mean - 2*std, (mean + 2*std)[::-1]]),
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=True,
        name='95% CI'
    ))

    # Guesses
    fig.add_trace(go.Scatter(
        x=results[dim_name],
        y=results['response'],
        mode='markers',
        marker=dict(color='red', size=point_size),
        name='Guesses'
    ))

    # Current Point
    fig.add_trace(go.Scatter(
        x=[coordinates[dim_name]],
        y=[simple_func(coordinates)],
        mode='markers',
        marker=dict(color='blue', size=14, symbol='x'),
        name='Current Point'
    ))

    fig.update_layout(
        title=f'GP along {dim_name} at point {coordinates}',
        xaxis_title=dim_name,
        yaxis_title='Response',
        legend=dict(x=1.05, y=0.5)
    )
    return fig

def plt_gp_all_dims_plotly(gp: SingleTaskGP, results_df: pd.DataFrame, coordinates: list[float]|Tensor):
    coordinates = coordinates.numpy() if isinstance(coordinates, Tensor) else coordinates
    dim_names = [col for col in results_df.columns if col != 'response']

    for i, dim_name in enumerate(dim_names):
        grid = torch.linspace(0, 20, 100)
        mean, std = GPVisualiser.get_plane_gaussian_for_point(gp, coordinates, fixed_dim=i, grid=grid)

        distances = [get_euclidean_distance(coordinates, row[:-1]) for row in results_df.to_numpy()]
        point_size = [(1 - d / max(distances)) * 20 + 8 for d in distances]  # Scale sizes

        fig = plt_gp_along_dimension_plotly(grid, mean, std, results_df, dim_name, coordinates, point_size)
        fig.show()

for point in results.itertuples():
    if point.Index < 18:
        continue  # Skip to later points for more interesting plots

    coordinates = [point.x0, point.x1, point.x2]
    plt_gp_all_dims_plotly(gp, results, coordinates)



NameError: name 'gp' is not defined